In [1]:
import pandas as pd
import os
import glob
VERBOSE=True
summary_cols= ['GEAR_CALIB_P (meas)', 'GEAR_CALIB_P (used)', 'GEAR_CALIB_T (meas)', 
               'GEAR_CALIB_T (used)',
 'LENGTH_R1 (meas)', 'LENGTH_R1 (used)', 'LENGTH_R2 (meas)', 'LENGTH_R2 (used)',
 'OFFSET_P (meas)', 'OFFSET_P (used)', 'OFFSET_T (meas)', 'OFFSET_T (used)', 'OFFSET_X (meas)',
 'OFFSET_X (used)', 'OFFSET_Y (meas)', 'OFFSET_Y (used)', 'PHYSICAL_RANGE_P (meas)', 
 'PHYSICAL_RANGE_P (used)', 'PHYSICAL_RANGE_T (meas)', 'PHYSICAL_RANGE_T (used)', 
 'bad data ignore this row (enter your initials and justification)', 
 'blind max (um) all targets', 'blind max (um) best 95%', 'calib mode', 'code version', 
 'corr max (um) all targets with 3.0 um threshold', 
 'corr max (um) all targets with 5.0 um threshold', 
 'corr max (um) best 95% with 3.0 um threshold', 
 'corr max (um) best 95% with 5.0 um threshold', 
 'corr rms (um) all targets with 3.0 um threshold', 
 'corr rms (um) all targets with 5.0 um threshold', 
 'corr rms (um) best 95% with 3.0 um threshold', 
 'corr rms (um) best 95% with 5.0 um threshold', 
 'curr creep', 'curr cruise', 'finish time', 'max num corr all targets with 3.0 um threshold',
 'max num corr all targets with 5.0 um threshold', 
 'max num corr best 95% with 3.0 um threshold', 
 'max num corr best 95% with 5.0 um threshold', 
 'mean num corr all targets with 3.0 um threshold', 
 'mean num corr all targets with 5.0 um threshold', 
 'mean num corr best 95% with 3.0 um threshold', 
 'mean num corr best 95% with 5.0 um threshold', 
 'num pts calib P', 'num pts calib T', 'num targets', 'operator notes', 'pos log files', 
 'ranges remeasured', 'relative humidity', 'start time', 'supply voltage', 
 'temperature (C)', 'test loop data file', 'test operator', 'test station', 
 'total limit seeks P at finish', 'total limit seeks T at finish', 
 'total move sequences at finish', 'xytest log file']
def merge(dataframe_list):
    """
            Use pandas to concatenate two dataframes, then drop duplicate rows 
            and finally sort by start time
    """
    m=pd.concat(dataframe_list) #,sort=False)
    m=m.drop_duplicates(subset=['start time'], keep='first')
    m=m.sort_values(by=['start time'],ascending=True)
    return m

def find_and_copy_summaries_not_in_targetpath(target_path, comparison_path):
    from shutil import copyfile


    target_file_list=[os.path.basename(f) for f in glob.glob(target_path+'/M0*_summary.csv')]
    comparison_file_list=[os.path.basename(f) for f in glob.glob(comparison_path+'/M0*_summary.csv')]
    ncopied=0
    #print(comparison_file_list)
    for cf in comparison_file_list:
        if cf not in target_file_list:
            print('Copying file:')
            print(os.path.join(comparison_path,cf)+'  --> '+os.path.join(target_path,cf))
            copyfile(os.path.join(comparison_path,cf), os.path.join(target_path,cf)) 
            ncopied =ncopied+1
    return ncopied

def find_and_merge_duplicates(target_path, comparison_path):
    '''
    Finds files on comparison_path that are also on target_path.
    Comparison is made by name strictly.
        
    '''  
    target_file_list=[os.path.basename(f) for f in glob.glob(target_path+'/M0*_summary.csv')]
    comparison_file_list=[os.path.basename(f) for f in glob.glob(comparison_path+'/M0*_summary.csv')]
    #if VERBOSE: print('target list',target_file_list)
    duplicates=list(set(target_file_list).intersection(comparison_file_list)) 
    #if VERBOSE: print('Duplicate files: ',duplicates)
    tot=0
    tot0=0
    tot1=0
    for k in duplicates:
        #print(k)
        #file_list=[target_path+k,comparison_path+k]
        try:
            df_list=[pd.read_csv(f,usecols=summary_cols) for f in [os.path.join(target_path,k),os.path.join(comparison_path,k)]]
            tot0=tot0+len(df_list[0].index)
            tot1=tot1+len(df_list[1].index)
            m=merge(df_list)
            #print('merged',len(m.index))
            tot=tot+len(m.index)
            os.rename(os.path.join(target_path,k),os.path.join(target_path,'old-'+k))
            m.to_csv(os.path.join(target_path,k))
        except Exception as e:            
            print('File: ',k)
            print('Error: ',e)
    print('Total:',tot0,tot1,tot)        
    return duplicates


def count_summary_entries(summary_file_name):
    '''
    returns the number of entries in a summary file
    '''
    df=pd.read_csv(summary_file_name)
    
    if len(df.index)==0: print(summary_file_name,len(df.index))
    return len(df.index)
    
def total_summary_entries(target_path):
    import collections
    total=[]
    file_list=glob.glob(os.path.join(target_path,'M0*'))
    for f in file_list:
        #print(f)
        total.append(count_summary_entries(f))
        #print(total)
    return collections.Counter(total)
def inspect_summary_file(filename):
    df=pd.read_csv(filename)#,usecols=summary_cols)
    return df

def fix_missing_column(filename):
    df=pd.read_csv(filename)
    for h in summary_cols:
        if h not in list(df): 
            print('missing column: ',h)
            df[h]=""
            print('fixed...')
            df.to_csv(filename)


In [34]:
target_path='/Users/michael/desi/data/posdata/summaries'
comp_path='//Users/michael/bucket'

#fname='M02859_summary.csv'
#n=count_summary_entries(os.path.join(target_path,fname))
#print(n)
#n=count_summary_entries(os.path.join(target_path,'old-'+fname))
#n=total_summary_entries(target_path)                   
#print(n)
#print(sum(n.values()))
#df=inspect_summary_file('/Users/michael/desi/poslogs/M00614_summary.csv')
#df2=inspect_summary_file('/Users/michael/desi/data/posdata/summaries/M00614_summary.csv')
#Users/michael/desi/data/posdata/summaries
files=['M00614_summary.csv','M00941_summary.csv','M00937_summary.csv','M00132_summary.csv',
'M00087_summary.csv','M00542_summary.csv','M00673_summary.csv','M00567_summary.csv','M00590_summary.csv',
'M00616_summary.csv','M00126_summary.csv','M00597_summary.csv','M00164_summary.csv',
'M00606_summary.csv','M00604_summary.csv','M00158_summary.csv','M00119_summary.csv','M00415_summary.csv',
'M00467_summary.csv','M00187_summary.csv','M00614_summary.csv','M00673_summary.csv','M00164_summary.csv']

FIX=True
DUB=True
TOTAL=True

if FIX:
    for f in files:
        print(f)
        try:
            fix_missing_column(os.path.join(target_path,f))
            fix_missing_column(os.path.join(comp_path,f))
        except Exception as e:
            print (e)

#print(list(df))
#x=list(set(list(df2))-set(list(summary_cols)))
#print(list(df2))
#print('stat')
#for h in summary_cols: #list(df2):
#   if h not in list(df): print(h)
#print(summary_cols)
#print(x)



if DUB:
    n=find_and_copy_summaries_not_in_targetpath(target_path, comp_path)
    print(n)
    dup=find_and_merge_duplicates(target_path, comp_path)

if TOTAL:
    n=total_summary_entries(target_path)                   
    print(n)

M00614_summary.csv
missing column:  bad data ignore this row (enter your initials and justification)
fixed...
M00941_summary.csv
missing column:  GEAR_CALIB_P (meas)
fixed...
missing column:  GEAR_CALIB_P (used)
fixed...
missing column:  GEAR_CALIB_T (meas)
fixed...
missing column:  GEAR_CALIB_T (used)
fixed...
missing column:  LENGTH_R1 (meas)
fixed...
missing column:  LENGTH_R1 (used)
fixed...
missing column:  LENGTH_R2 (meas)
fixed...
missing column:  LENGTH_R2 (used)
fixed...
missing column:  OFFSET_P (meas)
fixed...
missing column:  OFFSET_P (used)
fixed...
missing column:  OFFSET_T (meas)
fixed...
missing column:  OFFSET_T (used)
fixed...
missing column:  OFFSET_X (meas)
fixed...
missing column:  OFFSET_X (used)
fixed...
missing column:  OFFSET_Y (meas)
fixed...
missing column:  OFFSET_Y (used)
fixed...
missing column:  PHYSICAL_RANGE_P (meas)
fixed...
missing column:  PHYSICAL_RANGE_P (used)
fixed...
missing column:  PHYSICAL_RANGE_T (meas)
fixed...
missing column:  PHYSICAL_RAN

missing column:  bad data ignore this row (enter your initials and justification)
fixed...
M00119_summary.csv
missing column:  GEAR_CALIB_P (meas)
fixed...
missing column:  GEAR_CALIB_P (used)
fixed...
missing column:  GEAR_CALIB_T (meas)
fixed...
missing column:  GEAR_CALIB_T (used)
fixed...
missing column:  LENGTH_R1 (meas)
fixed...
missing column:  LENGTH_R1 (used)
fixed...
missing column:  LENGTH_R2 (meas)
fixed...
missing column:  LENGTH_R2 (used)
fixed...
missing column:  OFFSET_P (meas)
fixed...
missing column:  OFFSET_P (used)
fixed...
missing column:  OFFSET_T (meas)
fixed...
missing column:  OFFSET_T (used)
fixed...
missing column:  OFFSET_X (meas)
fixed...
missing column:  OFFSET_X (used)
fixed...
missing column:  OFFSET_Y (meas)
fixed...
missing column:  OFFSET_Y (used)
fixed...
missing column:  PHYSICAL_RANGE_P (meas)
fixed...
missing column:  PHYSICAL_RANGE_P (used)
fixed...
missing column:  PHYSICAL_RANGE_T (meas)
fixed...
missing column:  PHYSICAL_RANGE_T (used)
fixed..

In [35]:
x={13: 1502, 14: 684, 15: 397, 19: 380, 16: 269, 20: 117, 17: 114, 22: 91, 26: 64, 18: 62, 35: 37, 28: 32, 25: 29, 21: 28, 34: 28, 37: 26, 43: 24, 27: 22, 32: 20, 23: 19, 0: 18, 29: 18, 44: 14, 45: 14, 174: 14, 6: 13, 277: 12, 8: 11, 31: 10, 114: 9, 24: 9, 10: 8, 55: 8, 36: 7, 209: 7, 86: 7, 33: 7, 61: 7, 46: 6, 5: 6, 87: 6, 30: 6, 101: 5, 2: 5, 9: 5, 155: 5, 60: 5, 66: 5, 71: 4, 42: 3, 3: 3, 208: 3, 64: 3, 132: 3, 41: 2, 70: 2, 103: 2, 58: 2, 79: 2, 83: 2, 176: 2, 123: 2, 11: 2, 40: 2, 38: 2, 63: 2, 78: 2, 50: 2, 143: 2, 140: 2, 62: 2, 39: 2, 77: 1, 54: 1, 139: 1, 180: 1, 47: 1, 113: 1, 72: 1, 48: 1, 80: 1, 88: 1, 89: 1, 131: 1, 59: 1, 220: 1, 76: 1, 74: 1, 115: 1, 136: 1, 156: 1, 153: 1, 51: 1, 96: 1, 119: 1, 128: 1, 137: 1, 148: 1}
x={13: 4515, 14: 1080, 15: 500, 16: 300, 17: 128, 27: 98, 26: 92, 19: 86, 18: 60, 21: 45, 20: 40, 34: 35, 28: 30, 35: 29, 22: 27, 37: 26, 43: 25, 0: 22, 9: 20, 32: 17, 25: 16, 29: 14, 31: 14, 45: 14, 174: 14, 44: 13, 8: 13, 6: 13, 24: 12, 277: 12, 51: 11, 114: 9, 23: 8, 10: 8, 55: 8, 60: 8, 39: 8, 1: 7, 36: 7, 209: 7, 86: 7, 46: 6, 5: 6, 87: 6, 2: 6, 30: 6, 101: 5, 155: 5, 66: 5, 71: 4, 61: 4, 42: 3, 3: 3, 208: 3, 64: 3, 132: 3, 33: 3, 41: 2, 70: 2, 103: 2, 58: 2, 79: 2, 83: 2, 176: 2, 123: 2, 11: 2, 40: 2, 38: 2, 63: 2, 78: 2, 50: 2, 143: 2, 140: 2, 62: 2, 77: 1, 54: 1, 139: 1, 180: 1, 47: 1, 113: 1, 72: 1, 48: 1, 80: 1, 88: 1, 89: 1, 131: 1, 59: 1, 220: 1, 76: 1, 74: 1, 115: 1, 136: 1, 156: 1, 153: 1, 96: 1, 119: 1, 128: 1, 137: 1, 148: 1, 65: 1}
x={13: 3973, 14: 1132, 15: 541, 19: 380, 16: 288, 17: 134, 20: 117, 27: 96, 22: 91, 26: 91, 18: 62, 21: 48, 35: 37, 28: 32, 25: 29, 34: 28, 37: 26, 43: 24, 0: 22, 32: 20, 23: 19, 29: 18, 44: 14, 45: 14, 174: 14, 8: 13, 6: 13, 277: 12, 24: 11, 51: 11, 31: 10, 114: 9, 10: 8, 55: 8, 39: 8, 1: 7, 36: 7, 209: 7, 86: 7, 33: 7, 61: 7, 46: 6, 5: 6, 87: 6, 2: 6, 30: 6, 101: 5, 9: 5, 155: 5, 60: 5, 66: 5, 71: 4, 42: 3, 3: 3, 208: 3, 64: 3, 132: 3, 41: 2, 70: 2, 103: 2, 58: 2, 79: 2, 83: 2, 176: 2, 123: 2, 11: 2, 40: 2, 38: 2, 63: 2, 78: 2, 50: 2, 143: 2, 140: 2, 62: 2, 77: 1, 54: 1, 139: 1, 180: 1, 47: 1, 113: 1, 72: 1, 48: 1, 80: 1, 88: 1, 89: 1, 131: 1, 59: 1, 220: 1, 76: 1, 74: 1, 115: 1, 136: 1, 156: 1, 153: 1, 96: 1, 119: 1, 128: 1, 137: 1, 148: 1}
x={13: 3911, 14: 1095, 15: 541, 26: 406, 16: 304, 27: 179, 17: 161, 19: 82, 29: 78, 18: 65, 20: 47, 21: 43, 28: 42, 35: 41, 32: 28, 37: 28, 34: 28, 43: 24, 31: 23, 22: 21, 0: 21, 30: 17, 44: 14, 25: 14, 45: 14, 65: 14, 8: 13, 39: 13, 277: 12, 24: 10, 55: 9, 81: 9, 114: 9, 10: 8, 78: 8, 33: 8, 46: 7, 274: 7, 40: 7, 61: 7, 190: 7, 193: 7, 23: 6, 5: 6, 87: 6, 36: 6, 132: 6, 101: 5, 42: 5, 70: 5, 2: 5, 155: 5, 60: 5, 66: 5, 71: 4, 3: 3, 273: 3, 59: 3, 64: 3, 86: 3, 47: 3, 82: 2, 1: 2, 41: 2, 103: 2, 58: 2, 79: 2, 83: 2, 72: 2, 177: 2, 123: 2, 131: 2, 11: 2, 9: 2, 38: 2, 63: 2, 50: 2, 143: 2, 140: 2, 62: 2, 77: 1, 54: 1, 139: 1, 85: 1, 180: 1, 106: 1, 113: 1, 73: 1, 48: 1, 80: 1, 134: 1, 89: 1, 220: 1, 76: 1, 74: 1, 115: 1, 56: 1, 136: 1, 156: 1, 153: 1, 51: 1, 97: 1, 119: 1, 128: 1, 137: 1, 148: 1, 6: 1, 52: 1}
x={13: 3911, 14: 1095, 15: 541, 26: 406, 16: 304, 27: 179, 17: 161, 19: 82, 29: 78, 18: 65, 20: 47, 21: 43, 28: 42, 35: 41, 32: 28, 37: 28, 34: 28, 43: 24, 31: 23, 22: 21, 0: 21, 30: 17, 44: 14, 25: 14, 45: 14, 65: 14, 8: 13, 39: 13, 277: 12, 24: 10, 55: 9, 81: 9, 114: 9, 10: 8, 78: 8, 33: 8, 46: 7, 274: 7, 40: 7, 61: 7, 190: 7, 193: 7, 23: 6, 5: 6, 87: 6, 36: 6, 132: 6, 101: 5, 42: 5, 70: 5, 2: 5, 155: 5, 60: 5, 66: 5, 71: 4, 3: 3, 273: 3, 59: 3, 64: 3, 86: 3, 47: 3, 82: 2, 1: 2, 41: 2, 103: 2, 58: 2, 79: 2, 83: 2, 72: 2, 177: 2, 123: 2, 131: 2, 11: 2, 9: 2, 38: 2, 63: 2, 50: 2, 143: 2, 140: 2, 62: 2, 77: 1, 54: 1, 139: 1, 85: 1, 180: 1, 106: 1, 113: 1, 73: 1, 48: 1, 80: 1, 134: 1, 89: 1, 220: 1, 76: 1, 74: 1, 115: 1, 56: 1, 136: 1, 156: 1, 153: 1, 51: 1, 97: 1, 119: 1, 128: 1, 137: 1, 148: 1, 6: 1, 52: 1}
x={13: 3911, 14: 1095, 15: 541, 26: 406, 16: 304, 27: 179, 17: 161, 19: 82, 29: 78, 18: 65, 20: 47, 21: 43, 28: 42, 35: 41, 32: 28, 37: 28, 34: 28, 43: 24, 31: 23, 22: 21, 0: 21, 30: 17, 44: 14, 25: 14, 45: 14, 65: 14, 8: 13, 39: 13, 277: 12, 24: 10, 55: 9, 81: 9, 114: 9, 10: 8, 78: 8, 33: 8, 46: 7, 274: 7, 40: 7, 61: 7, 190: 7, 193: 7, 23: 6, 5: 6, 87: 6, 36: 6, 132: 6, 101: 5, 42: 5, 70: 5, 2: 5, 155: 5, 60: 5, 66: 5, 71: 4, 3: 3, 273: 3, 59: 3, 64: 3, 86: 3, 47: 3, 82: 2, 1: 2, 41: 2, 103: 2, 58: 2, 79: 2, 83: 2, 72: 2, 177: 2, 123: 2, 131: 2, 11: 2, 9: 2, 38: 2, 63: 2, 50: 2, 143: 2, 140: 2, 62: 2, 77: 1, 54: 1, 139: 1, 85: 1, 180: 1, 106: 1, 113: 1, 73: 1, 48: 1, 80: 1, 134: 1, 89: 1, 220: 1, 76: 1, 74: 1, 115: 1, 56: 1, 136: 1, 156: 1, 153: 1, 51: 1, 97: 1, 119: 1, 128: 1, 137: 1, 148: 1, 6: 1, 52: 1}
x={13: 3911, 14: 1095, 15: 541, 26: 406, 16: 304, 27: 179, 17: 161, 19: 82, 29: 78, 18: 65, 20: 47, 21: 43, 28: 42, 35: 41, 32: 28, 37: 28, 34: 28, 43: 24, 31: 23, 22: 21, 0: 21, 30: 17, 44: 14, 25: 14, 45: 14, 65: 14, 8: 13, 39: 13, 277: 12, 24: 10, 55: 9, 81: 9, 114: 9, 10: 8, 78: 8, 33: 8, 46: 7, 274: 7, 40: 7, 61: 7, 190: 7, 193: 7, 23: 6, 5: 6, 87: 6, 36: 6, 132: 6, 101: 5, 42: 5, 70: 5, 2: 5, 155: 5, 60: 5, 66: 5, 71: 4, 3: 3, 273: 3, 59: 3, 64: 3, 86: 3, 47: 3, 82: 2, 1: 2, 41: 2, 103: 2, 58: 2, 79: 2, 83: 2, 72: 2, 177: 2, 123: 2, 131: 2, 11: 2, 9: 2, 38: 2, 63: 2, 50: 2, 143: 2, 140: 2, 62: 2, 77: 1, 54: 1, 139: 1, 85: 1, 180: 1, 106: 1, 113: 1, 73: 1, 48: 1, 80: 1, 134: 1, 89: 1, 220: 1, 76: 1, 74: 1, 115: 1, 56: 1, 136: 1, 156: 1, 153: 1, 51: 1, 97: 1, 119: 1, 128: 1, 137: 1, 148: 1, 6: 1, 52: 1}
x={13: 3920, 14: 1099, 15: 541, 26: 407, 16: 304, 27: 179, 17: 160, 19: 82, 29: 78, 18: 65, 20: 47, 21: 43, 28: 42, 35: 41, 32: 28, 34: 28, 37: 27, 43: 24, 31: 23, 22: 21, 0: 21, 30: 18, 39: 16, 44: 15, 25: 14, 45: 14, 65: 14, 8: 13, 277: 12, 40: 11, 24: 10, 55: 9, 81: 9, 114: 9, 10: 8, 78: 8, 33: 8, 46: 7, 274: 7, 61: 7, 190: 7, 193: 7, 23: 6, 5: 6, 87: 6, 36: 6, 132: 6, 101: 5, 70: 5, 2: 5, 155: 5, 60: 5, 66: 5, 42: 4, 71: 4, 3: 3, 273: 3, 59: 3, 64: 3, 86: 3, 47: 3, 82: 2, 1: 2, 41: 2, 103: 2, 58: 2, 79: 2, 83: 2, 72: 2, 177: 2, 123: 2, 131: 2, 11: 2, 9: 2, 38: 2, 63: 2, 50: 2, 143: 2, 140: 2, 62: 2, 77: 1, 54: 1, 139: 1, 85: 1, 180: 1, 106: 1, 113: 1, 73: 1, 48: 1, 80: 1, 134: 1, 89: 1, 220: 1, 76: 1, 74: 1, 115: 1, 56: 1, 136: 1, 156: 1, 153: 1, 51: 1, 97: 1, 119: 1, 128: 1, 137: 1, 148: 1, 6: 1, 52: 1}
x={13: 3920, 14: 1099, 15: 541, 26: 407, 16: 304, 27: 179, 17: 160, 19: 82, 29: 78, 18: 65, 20: 47, 21: 43, 28: 42, 35: 41, 32: 28, 34: 28, 37: 27, 43: 24, 31: 23, 22: 21, 0: 21, 30: 18, 39: 16, 44: 15, 25: 14, 45: 14, 65: 14, 8: 13, 277: 12, 40: 11, 24: 10, 55: 9, 81: 9, 114: 9, 10: 8, 78: 8, 33: 8, 46: 7, 274: 7, 61: 7, 190: 7, 193: 7, 23: 6, 5: 6, 87: 6, 36: 6, 132: 6, 101: 5, 70: 5, 2: 5, 155: 5, 60: 5, 66: 5, 42: 4, 71: 4, 3: 3, 273: 3, 59: 3, 64: 3, 86: 3, 47: 3, 82: 2, 1: 2, 41: 2, 103: 2, 58: 2, 79: 2, 83: 2, 72: 2, 177: 2, 123: 2, 131: 2, 11: 2, 9: 2, 38: 2, 63: 2, 50: 2, 143: 2, 140: 2, 62: 2, 77: 1, 54: 1, 139: 1, 85: 1, 180: 1, 106: 1, 113: 1, 73: 1, 48: 1, 80: 1, 134: 1, 89: 1, 220: 1, 76: 1, 74: 1, 115: 1, 56: 1, 136: 1, 156: 1, 153: 1, 51: 1, 97: 1, 119: 1, 128: 1, 137: 1, 148: 1, 6: 1, 52: 1}
x={13: 3920, 14: 1099, 15: 541, 26: 407, 16: 304, 27: 179, 17: 160, 19: 82, 29: 78, 18: 65, 20: 47, 21: 43, 28: 42, 35: 41, 32: 28, 34: 28, 37: 27, 43: 24, 31: 23, 22: 21, 0: 21, 30: 18, 39: 16, 44: 15, 25: 14, 45: 14, 65: 14, 8: 13, 277: 12, 40: 11, 24: 10, 55: 9, 81: 9, 114: 9, 10: 8, 78: 8, 33: 8, 46: 7, 274: 7, 61: 7, 190: 7, 193: 7, 23: 6, 5: 6, 87: 6, 36: 6, 132: 6, 101: 5, 70: 5, 2: 5, 155: 5, 60: 5, 66: 5, 42: 4, 71: 4, 3: 3, 273: 3, 59: 3, 64: 3, 86: 3, 47: 3, 82: 2, 1: 2, 41: 2, 103: 2, 58: 2, 79: 2, 83: 2, 72: 2, 177: 2, 123: 2, 131: 2, 11: 2, 9: 2, 38: 2, 63: 2, 50: 2, 143: 2, 140: 2, 62: 2, 77: 1, 54: 1, 139: 1, 85: 1, 180: 1, 106: 1, 113: 1, 73: 1, 48: 1, 80: 1, 134: 1, 89: 1, 220: 1, 76: 1, 74: 1, 115: 1, 56: 1, 136: 1, 156: 1, 153: 1, 51: 1, 97: 1, 119: 1, 128: 1, 137: 1, 148: 1, 6: 1, 52: 1}

sorted_by_value = sorted(x.items(), key=lambda kv: kv[0])
print(sorted_by_value)
t=[a[0]*a[1] for a in sorted_by_value]
print(sum(t))

[(0, 21), (1, 2), (2, 5), (3, 3), (5, 6), (6, 1), (8, 13), (9, 2), (10, 8), (11, 2), (13, 3920), (14, 1099), (15, 541), (16, 304), (17, 160), (18, 65), (19, 82), (20, 47), (21, 43), (22, 21), (23, 6), (24, 10), (25, 14), (26, 407), (27, 179), (28, 42), (29, 78), (30, 18), (31, 23), (32, 28), (33, 8), (34, 28), (35, 41), (36, 6), (37, 27), (38, 2), (39, 16), (40, 11), (41, 2), (42, 4), (43, 24), (44, 15), (45, 14), (46, 7), (47, 3), (48, 1), (50, 2), (51, 1), (52, 1), (54, 1), (55, 9), (56, 1), (58, 2), (59, 3), (60, 5), (61, 7), (62, 2), (63, 2), (64, 3), (65, 14), (66, 5), (70, 5), (71, 4), (72, 2), (73, 1), (74, 1), (76, 1), (77, 1), (78, 8), (79, 2), (80, 1), (81, 9), (82, 2), (83, 2), (85, 1), (86, 3), (87, 6), (89, 1), (97, 1), (101, 5), (103, 2), (106, 1), (113, 1), (114, 9), (115, 1), (119, 1), (123, 2), (128, 1), (131, 2), (132, 6), (134, 1), (136, 1), (137, 1), (139, 1), (140, 2), (143, 2), (148, 1), (153, 1), (155, 5), (156, 1), (177, 2), (180, 1), (190, 7), (193, 7), (220, 1

In [29]:
import collections
summary_path='/Users/michael/desi/data/posdata/summaries/'
movedata_path='/Users/michael/desi/data/posdata/movedata/'

sum_file_list = glob.glob(summary_path+'M0*_summary.csv')
mvdata_file_list= glob.glob(movedata_path+'M0*_movedata.csv')
mvdata_pos_list=[os.path.basename(mf)[:6] for mf in mvdata_file_list]
m=collections.Counter(mvdata_pos_list)
print(m)
for sf in sum_file_list[2160:2250]:    
    pos=os.path.basename(sf).rstrip('_summary.csv')
    df=pd.read_csv(sf)
    print(pos,len(df.index), m[pos] )
    


Counter({'M00892': 114, 'M00695': 110, 'M00716': 107, 'M00719': 107, 'M00722': 107, 'M00723': 107, 'M00726': 107, 'M00713': 106, 'M00727': 106, 'M00004': 105, 'M00176': 90, 'M05226': 79, 'M05227': 79, 'M05228': 79, 'M05229': 79, 'M05230': 79, 'M05231': 79, 'M05232': 79, 'M05234': 79, 'M05247': 79, 'M05248': 79, 'M05249': 79, 'M05250': 79, 'M05251': 79, 'M05256': 79, 'M00763': 75, 'M00764': 75, 'M01026': 72, 'M00672': 59, 'M03116': 29, 'M05003': 23, 'M05924': 16, 'M05925': 16, 'M05926': 16, 'M05927': 16, 'M05928': 16, 'M05929': 16, 'M05930': 16, 'M05931': 16, 'M06024': 16, 'M06025': 16, 'M03117': 15, 'M03118': 15, 'M03119': 15, 'M03121': 15, 'M03124': 15, 'M02164': 14, 'M02702': 14, 'M02754': 14, 'M02812': 14, 'M02813': 14, 'M02815': 14, 'M02915': 14, 'M02935': 14, 'M03001': 14, 'M03002': 14, 'M03006': 14, 'M03025': 14, 'M03032': 14, 'M03064': 14, 'M03080': 14, 'M03093': 14, 'M03110': 14, 'M03111': 14, 'M03112': 14, 'M03113': 14, 'M03114': 14, 'M03115': 14, 'M03120': 14, 'M03122': 14, '

M02886 14 0
M02887 26 0
M02888 14 0
M02889 14 0
M02890 14 0
M02891 14 0
M02892 14 0
M02893 27 10
M02894 26 0
M02895 13 0
M02896 26 10
M02897 16 0
M02898 14 0
M02899 16 0
M02900 16 0
M02901 17 1
M02902 17 1
M02903 16 0
M02904 17 1
M02905 16 0
M02906 17 1
M02907 29 10
M02908 29 10
M02909 29 10
M02910 17 0
M02911 15 0
M02912 15 0
M02913 17 1
M02914 26 10
M02915 29 14
M02916 13 0
M02917 13 0
M02918 28 10
M02919 16 1
M02920 15 0
M02921 15 0
M02922 16 1
M02923 13 0


In [ ]:
import glob
# main
# Find and merge summary files with identical positioner ID (in the same path)
# Those files are created due to problems with SVN committ / update
# See function 'find_multiple' for an example
# Adjust the path below to fit your needs.
# The merged summary file will have a filename ending in '_summary_merged.csv'
# The original files are kept intact.
VERBOSE=True
path='/Users/michael/desi/poslogs/summaries/'
not_unique=find_multiple(path,'M05206_summary.csv*')

if VERBOSE: print(not_unique.keys())

for k in not_unique.keys():
    if VERBOSE: print('posid: '+str(k))
    file_list=not_unique[k]
    if VERBOSE:
        for f in file_list: print(f)
    df_list=[pd.read_csv(f) for f in file_list]
    m=merge(df_list)
    new_filename=file_list[0][0:file_list[0].find(k)]+k+'_summary_merged.csv'
    if VERBOSE: print(new_filename)
    m.to_csv(new_filename)

In [ ]:
import pandas as pd
import glob
import shutil
import sys
import os
from os import path
#
# colum list for summary files
summary_cols= ['GEAR_CALIB_P (meas)', 'GEAR_CALIB_P (used)', 'GEAR_CALIB_T (meas)', 
               'GEAR_CALIB_T (used)',
 'LENGTH_R1 (meas)', 'LENGTH_R1 (used)', 'LENGTH_R2 (meas)', 'LENGTH_R2 (used)',
 'OFFSET_P (meas)', 'OFFSET_P (used)', 'OFFSET_T (meas)', 'OFFSET_T (used)', 'OFFSET_X (meas)',
 'OFFSET_X (used)', 'OFFSET_Y (meas)', 'OFFSET_Y (used)', 'PHYSICAL_RANGE_P (meas)', 
 'PHYSICAL_RANGE_P (used)', 'PHYSICAL_RANGE_T (meas)', 'PHYSICAL_RANGE_T (used)', 
 'bad data ignore this row (enter your initials and justification)', 
 'blind max (um) all targets', 'blind max (um) best 95%', 'calib mode', 'code version', 
 'corr max (um) all targets with 3.0 um threshold', 
 'corr max (um) all targets with 5.0 um threshold', 
 'corr max (um) best 95% with 3.0 um threshold', 
 'corr max (um) best 95% with 5.0 um threshold', 
 'corr rms (um) all targets with 3.0 um threshold', 
 'corr rms (um) all targets with 5.0 um threshold', 
 'corr rms (um) best 95% with 3.0 um threshold', 
 'corr rms (um) best 95% with 5.0 um threshold', 
 'curr creep', 'curr cruise', 'finish time', 'max num corr all targets with 3.0 um threshold',
 'max num corr all targets with 5.0 um threshold', 
 'max num corr best 95% with 3.0 um threshold', 
 'max num corr best 95% with 5.0 um threshold', 
 'mean num corr all targets with 3.0 um threshold', 
 'mean num corr all targets with 5.0 um threshold', 
 'mean num corr best 95% with 3.0 um threshold', 
 'mean num corr best 95% with 5.0 um threshold', 
 'num pts calib P', 'num pts calib T', 'num targets', 'operator notes', 'pos log files', 
 'ranges remeasured', 'relative humidity', 'start time', 'supply voltage', 
 'temperature (C)', 'test loop data file', 'test operator', 'test station', 
 'total limit seeks P at finish', 'total limit seeks T at finish', 
 'total move sequences at finish', 'xytest log file']

# main
# Find and merge summary files with identical positioner ID (in the same path)
# Those files are created due to problems with SVN committ / update
# See function 'find_multiple' for an example
# Adjust the path below to fit your needs.
# The merged summary file will have a filename ending in '_summary_merged.csv'
# The original files are kept intact.
VERBOSE=True
selector='M0????_summary.csv'
path1='/Users/michael/desi/posdata/summaries/'
path2='/Users/michael/desi/posdata/summaries/1/'


'''
df=pd.read_csv(path1+'M00087_summary.csv') #,usecols=summary_cols)

#print(list(df))
l1=list(df)
df=pd.read_csv(path2+'M00087_summary.csv')#,usecols=summary_cols)
l2=list(df)
print(list(df))
print (set(l1)-set(l2))
print (set(l2)-set(summary_cols))
'''
#
# path1 has the master files
# path2 has the files we want to merge in
#
# create a list of all master files (full path)
master_list=glob.glob(path1+selector)
# create a list of all files in the master list (file names only)
master_list_filenames=[path.basename(f) for f in master_list]
# create a list of merge files (full path)
merge_list=glob.glob(path2+selector)
merge_list_filenames=[path.basename(f) for f in merge_list]
#
# copy files from merge list that are in master list to path1
# print out a warning if a file is only in the merge list
#

for f in merge_list_filenames:
    if f in master_list_filenames:
        #pass
        #print(path2+f, path1+f+'.1')
        shutil.copy(path2+f, path1+f+'.1')
    else:
        print("no match in master list",f)


for file in master_list_filenames:
    print(file)
    not_unique=find_multiple(path1, file+'*')

    #if VERBOSE: print('>>>',not_unique.keys())
    
    for k in not_unique.keys():
        if VERBOSE: print('posid: '+str(k))
        file_list=not_unique[k]
        if VERBOSE:
            for f in file_list: print(f)
                
                #list(set(list1).intersection(list2))
                
                #usecols=list( set(list(pd.read_csv(f))).intersection(summary_cols))
        df_list=[pd.read_csv(f,usecols=list( set(list(pd.read_csv(f))).intersection(summary_cols))) for f in file_list]
        m=merge(df_list)
        head=file_list[0][0:file_list[0].find(k)]+k
        new_filename=head+'_summary_merged.csv'
        if VERBOSE: print(new_filename)
        m.to_csv(new_filename,index=False)
        
        os.rename(head+'_summary.csv',head+'_summary_old_master.csv')
        os.rename(head+'_summary_merged.csv',head+'_summary.csv')
        os.remove(head+'_summary_old_master.csv')
        os.remove(head+'_summary.csv.1')
        #assert False, "breakpoint"
print('done')

In [ ]:
def find_multiple(path,name):
    from collections import Counter
    '''
    finds all summary files in the path that are not posid-unique
    Input: 
        path: path to check
    Returns: 
        not_unique: dictionary with positioner IDs that are not unique (i.e. multiple files exist) and 
                    list of files
    Example:'/Volumes/Saturn/data/summaries/M03691_summary.csv'
            '/Volumes/Saturn/data/summaries/M03692_summary.csv'
            '/Volumes/Saturn/data/summaries/M03692_summary.csv.mine'
            '/Volumes/Saturn/data/summaries/M03692_summary.csv.r115418'
            '/Volumes/Saturn/data/summaries/M03693_summary.csv'
            '/Volumes/Saturn/data/summaries/M03694_summary.csv'
    
        The dictionary {'M03692':['/Volumes/Saturn/data/summaries/M03692_summary.csv',
        '/Volumes/Saturn/data/summaries/M03692_summary.csv.mine',
        '/Volumes/Saturn/data/summaries/M03692_summary.csv.r115418]] will be returned.
    '''
    not_unique={}
    file_list=glob.glob(os.path.join(path,name))
    print(file_list)
    posids=[f.lstrip(path)[:6] for f in file_list]
    not_unique_list=[k for (k,v) in Counter(posids).items() if v > 1]
    for n in not_unique_list:
        files=[k for k in file_list if k[:len(path)+6]==path+n]
        not_unique[n]=files
    return not_uniqu